# 🎬 Turtle App: Multi-Agent Home Theater Assistant

## Simple demonstration of LangGraph + RAG system

This notebook showcases the Turtle App's multi-agent system for home theater management, featuring:
- **LangGraph multi-agent orchestration**
- **RAG-powered movie database** (42,000+ movies)
- **Specialized agents** for different tasks
- **Conversation memory** for multi-turn interactions

Let's explore the system through live code examples!

## 1. 🚀 Introduction & Setup

In [1]:
# Setup imports
import sys
import os

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

from turtleapp.src.workflows.graph import movie_workflow_agent
from turtleapp.src.core.tools.movie_summaries_retriever import movie_retriever_tool

print("🎬 Turtle App Multi-Agent System Ready")
print("📊 System Components:")
print("  - Supervisor: Claude 3.5 Sonnet (routing decisions)")
print("  - Agents: Claude 3.5 Haiku (specialized tasks)")
print("  - RAG Database: 42,000+ movies in Pinecone")
print("  - Tools: Movie retriever, Library scanner, Torrent manager")

2025-07-30 21:16:45,314 - supervisor.py:18 - INFO - Initializing SupervisorNodeCreator with members: ['movie_details_retriever_agent', 'movies_download_manager', 'library_manager_agent']


🎬 Turtle App Multi-Agent System Ready
📊 System Components:
  - Supervisor: Claude 3.5 Sonnet (routing decisions)
  - Agents: Claude 3.5 Haiku (specialized tasks)
  - RAG Database: 42,000+ movies in Pinecone
  - Tools: Movie retriever, Library scanner, Torrent manager


## 2. 🔍 Direct Tool Testing

Let's test the RAG movie retriever directly to see how it searches through our movie database:

In [2]:
# Test the RAG movie retriever directly
query = "sci-fi movies about artificial intelligence"
result = movie_retriever_tool._run(query, max_results=3)
print(f"Query: {query}")
print(f"Results:\n{result}")

2025-07-30 21:16:46,540 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Query: sci-fi movies about artificial intelligence
Results:
Found 3 movies matching 'sci-fi movies about artificial intelligence':

1. A.I. Artificial Intelligence (2001)
   Director: Steven Spielberg
   Cast: Jude Law, Haley Joel Osment, William Hurt, Frances O'Connor, Sam Robards
   Genre: science fiction
   Plot: In the late 22nd century, rising sea levels from global warming have wiped out coastal cities such as Amsterdam, Venice, and New York, and drastically reduced the world's population. A new type of robots called Mecha, advanced humanoids capable of thoughts and emotions, have been created.
David, a Mecha that resembles a human child and is programmed to display love for its owners, is sent to Henry Swinton, and his wife, Monica, as a replacement for their son, Martin, who has been placed in suspended animation until he can be cured of a rare disease. Monica warms to David and activates his imprinting protocol, causing him to have an enduring childlike love for her. David is 

In [3]:
# Test with different queries
queries = [
    "romantic comedies from the 90s",
    "Tom Hanks dramatic films", 
    "thriller movies with plot twists"
]

for q in queries:
    print(f"\n--- Query: {q} ---")
    result = movie_retriever_tool._run(q, max_results=2)
    print(result)


--- Query: romantic comedies from the 90s ---


2025-07-30 21:16:48,413 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Found 2 movies matching 'romantic comedies from the 90s':

1. The Romantics (2010)
   Director: Galt Niederhoffer
   Cast: Katie Holmes, Josh Duhamel, Anna Paquin, Malin Åkerman, Adam Brody, Dianna Agron, Jeremy Strong, Rebecca Lawrence, Candice Bergen, Elijah Wood
   Genre: romantic comedy
   Plot: A group of seven college friends reunite after six years for a wedding. Things go awry when the maid of honor, Laura, (Katie Holmes[3]) and the bride, Lila (Anna Paquin), clash over the groom, Tom (Josh Duhamel), with whom Laura was once romantically involved. As Laura, Lila, and Tom all try to decipher their emotions, the film explores all of the relationships of people in and around the circle of friends that met those years ago.

2. The Sweetest Thing (2002)
   Director: Roger Kumble
   Cast: Cameron Diaz, Christina Applegate, Selma Blair
   Genre: comedy
   Plot: In an opening scene, a group of men are interviewed regarding Christina Walters (Cameron Diaz) who introduce her as a player 

2025-07-30 21:16:49,245 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Found 2 movies matching 'Tom Hanks dramatic films':

1. Captain Phillips (2013)
   Director: Paul Greengrass
   Cast: Tom Hanks, Barkhad Abdi, Catherine Keener, Max Martini, Yul Vazquez, Michael Chernus, Chris Mulkey, Corey Johnson, David Warshofsky, John Magaro, Angus MacInnes
   Genre: action drama, bio-pic, thriller
   Plot: Richard Phillips takes command of MV Maersk Alabama, an unarmed container ship from the Port of Salalah in Oman, with orders to sail through the Gulf of Aden to Mombasa, Kenya. Wary of pirate activity off the coast of the Horn of Africa, he and First Officer Shane Murphy order strict security precautions on the vessel and carry out practice drills. During a drill, the vessel is chased by Somali pirates in two skiffs, and Phillips calls for help. Knowing that the pirates are listening to radio traffic, he pretends to call a warship, requesting immediate air support. One skiff turns around in response, and the other – manned by four heavily armed pirates led by Ab

2025-07-30 21:16:51,170 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Found 2 movies matching 'thriller movies with plot twists':

1. Twisted (2004)
   Director: Philip Kaufman
   Cast: Ashley Judd, Samuel L. Jackson, Andy García
   Genre: thriller
   Plot: Having solved a high-profile case involving serial killer Edmund Cutler that ended with her being taken hostage by Cutler but managing to overpower and arrest him regardless, Jessica Shepard (Ashley Judd) is a rising officer in the San Francisco Police Department. She is transferred to the homicide division and promoted to the rank of inspector. Her deceased father's former partner, SFPD Chief John Mills (Samuel L. Jackson), her foster father, also serves as her proud mentor. Shepard finds that she might once again have to prove herself in a department that takes no prisoners. In addition, Shepard's parents were both killed in the 1970s when she was young as a result of her father murdering her mother's extra-marital lovers then himself and Shepard's mother, an event which scarred her in t...

2. Dist

## 3. 🤖 Multi-Agent Workflow Examples

Now let's see the full multi-agent system in action. The supervisor will route different requests to specialized agents:

In [4]:
# Simple workflow demonstrations
test_messages = [
    "Tell me about Terminator 2",
    "What movies are in my library?",
    "Search for Matrix downloads",
    "Find me action movies from the 80s"
]

for message in test_messages:
    print(f"\n🎬 User: {message}")
    result, thread_id = movie_workflow_agent.invoke(message)
    print(f"🤖 Assistant: {result['messages'][-1].content}")
    print(f"Thread ID: {thread_id}")

2025-07-30 21:16:51,806 - supervisor.py:21 - INFO - Supervisor processing request



🎬 User: Tell me about Terminator 2


2025-07-30 21:16:53,617 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:16:53,632 - supervisor.py:34 - INFO - Supervisor routing to: movie_details_retriever_agent
2025-07-30 21:16:54,272 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:16:55,951 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:16:57,219 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:01,377 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17:02,821 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:09,192 - supervisor.py:21 - INFO - Supervisor processing request
2025-07-30 21:17:10,826 - _client.py:1025 - INFO - HTTP Req

🤖 Assistant: Agent stopped due to iteration limit or time limit.
Thread ID: 20250730_211651_565e080c

🎬 User: What movies are in my library?


2025-07-30 21:17:12,611 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:12,613 - supervisor.py:34 - INFO - Supervisor routing to: library_manager_agent
2025-07-30 21:17:12,615 - connection.py:741 - INFO - Initialising connection, guid: 02f04da2-3c89-4f04-9f95-41bbd4c0cd53, require_signing: True, server_name: 192.168.1.205, port: 445
2025-07-30 21:17:12,616 - connection.py:867 - INFO - Setting up transport connection
2025-07-30 21:17:12,616 - transport.py:64 - INFO - Connecting to DirectTcp socket
2025-07-30 21:17:12,624 - connection.py:876 - INFO - Starting negotiation with SMB server
2025-07-30 21:17:12,626 - connection.py:1546 - INFO - Negotiating with SMB2 protocol with highest client dialect of: SMB_3_1_1
2025-07-30 21:17:12,629 - connection.py:1610 - INFO - Sending SMB2 Negotiate message
2025-07-30 21:17:12,635 - connection.py:1615 - INFO - Receiving SMB2 Negotiate response
2025-07-30 21:17:12,636 - connection.

🤖 Assistant: Library scan completed. Found 2 movies.
File types: .mkv: 2
Sample movies:
- DieHart2DieHarter2024
- Hardcore Henry 2015

Thread ID: 20250730_211710_ed9e8a0c

🎬 User: Search for Matrix downloads


2025-07-30 21:17:16,537 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:16,541 - supervisor.py:34 - INFO - Supervisor routing to: movies_download_manager
2025-07-30 21:17:17,217 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:22,859 - error_handler.py:62 - ERROR - TorrentAPI error in api_call: 400 Client Error: Bad Request for url: http://192.168.1.205:15080/api/v2/search/results
2025-07-30 21:17:22,861 - error_handler.py:62 - ERROR - TorrentAPI error in search_torrents: 'NoneType' object has no attribute 'status_code'
2025-07-30 21:17:23,589 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:29,170 - error_handler.py:62 - ERROR - TorrentAPI error in api_call: 400 Client Error: Bad Request for url: http://192.168.1.205:15080/api/v2/search/results
2025-07-30 21:17:29,172 - error_h

🤖 Assistant: Agent stopped due to iteration limit or time limit.
Thread ID: 20250730_211714_428cacc4

🎬 User: Find me action movies from the 80s


2025-07-30 21:17:37,172 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:37,178 - supervisor.py:34 - INFO - Supervisor routing to: movie_details_retriever_agent
2025-07-30 21:17:38,689 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:40,485 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17:41,742 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:43,903 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17:45,302 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:47,375 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17

🤖 Assistant: Agent stopped due to iteration limit or time limit.
Thread ID: 20250730_211734_b7614853


## 4. 💬 Conversation Memory Demo

The system maintains conversation context across multiple turns using the same thread ID:

In [4]:
# Multi-turn conversation with same thread
thread_id = None

conversation = [
    "Tell me about The Matrix",
    "What about the sequels?",
    "Do I have any of these movies locally?",
    "Can you find similar sci-fi films?"
]

for msg in conversation:
    print(f"\n🎬 User: {msg}")
    result, thread_id = movie_workflow_agent.invoke(msg, thread_id=thread_id)
    print(f"🤖 Assistant: {result['messages'][-1].content}")

2025-07-30 21:17:49,057 - supervisor.py:21 - INFO - Supervisor processing request



🎬 User: Tell me about The Matrix


2025-07-30 21:17:50,465 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:50,473 - supervisor.py:34 - INFO - Supervisor routing to: movie_details_retriever_agent
2025-07-30 21:17:51,247 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:52,885 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17:54,105 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:17:57,154 - _client.py:1025 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 21:17:58,340 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:06,507 - supervisor.py:21 - INFO - Supervisor processing request
2025-07-30 21:18:08,607 - _client.py:1025 - INFO - HTTP Req

🤖 Assistant: Agent stopped due to iteration limit or time limit.

🎬 User: What about the sequels?


2025-07-30 21:18:10,708 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:10,716 - supervisor.py:34 - INFO - Supervisor routing to: movie_details_retriever_agent
2025-07-30 21:18:11,746 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:16,043 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:21,359 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:22,991 - supervisor.py:21 - INFO - Supervisor processing request
2025-07-30 21:18:24,533 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:24,537 - supervisor.py:32 - INFO - Supervisor routing to END
2025-07-30 21:18:24,542 - supervisor.py:21 - INFO - Supervisor processing request


...

2025-07-30 21:18:25,868 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:25,875 - supervisor.py:34 - INFO - Supervisor routing to: library_manager_agent
2025-07-30 21:18:25,878 - open.py:1163 - INFO - Session: dave, Tree Connect: \\192.168.1.205\daves - receiving SMB2 Create Response
2025-07-30 21:18:25,884 - open.py:1428 - INFO - Session: dave, Tree Connect: \\192.168.1.205\daves - sending SMB2 Query Directory Request for directory elements_main\torrent\incomplete
2025-07-30 21:18:25,885 - open.py:1439 - INFO - Session: dave, Tree Connect: \\192.168.1.205\daves - receiving SMB2 Query Response
2025-07-30 21:18:25,890 - open.py:1428 - INFO - Session: dave, Tree Connect: \\192.168.1.205\daves - sending SMB2 Query Directory Request for directory elements_main\torrent\incomplete
2025-07-30 21:18:25,891 - open.py:1439 - INFO - Session: dave, Tree Connect: \\192.168.1.205\daves - receiving SMB2 Query Response
2025-07-30 21

🤖 Assistant: Library scan completed. Found 2 movies.
File types: .mkv: 2
Sample movies:
- DieHart2DieHarter2024
- Hardcore Henry 2015


🎬 User: Can you find similar sci-fi films?


2025-07-30 21:18:28,683 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:28,688 - supervisor.py:34 - INFO - Supervisor routing to: movie_details_retriever_agent
2025-07-30 21:18:29,654 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:34,296 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:41,123 - _client.py:1025 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-07-30 21:18:44,297 - supervisor.py:21 - INFO - Supervisor processing request


KeyboardInterrupt: 

KeyboardInterrupt: 

## 5. 🎯 Agent Specialization Examples

Different types of requests get routed to different specialized agents:

In [ ]:
# Show different agents being selected
agent_examples = {
    "Movie questions": "What's the plot of Inception?",
    "Library management": "Scan my movie collection", 
    "Download management": "Check my download status",
    "Movie search": "Find torrents for The Dark Knight"
}

for category, question in agent_examples.items():
    print(f"\n--- {category} ---")
    print(f"Query: {question}")
    result, _ = movie_workflow_agent.invoke(question)
    response = result['messages'][-1].content
    # Truncate long responses for display
    display_response = response[:200] + "..." if len(response) > 200 else response
    print(f"Response: {display_response}")

## 6. 🎮 Interactive Playground

Try the interactive chat interface to have a real conversation with the Turtle App:

In [ ]:
# Simple interactive loop
def chat_with_turtle_app():
    print("🎬 Chat with Turtle App (type 'quit' to exit)")
    thread_id = None
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'quit':
            break
            
        result, thread_id = movie_workflow_agent.invoke(user_input, thread_id=thread_id)
        print(f"Assistant: {result['messages'][-1].content}")

# Uncomment to run interactive chat
# chat_with_turtle_app()

## 7. 📊 System Information

Let's explore the system components and database capabilities:

In [ ]:
# Show system components
print("🏗️ System Architecture:")
print("- Supervisor: Claude 3.5 Sonnet (routing decisions)")
print("- Agents: Claude 3.5 Haiku (specialized tasks)")
print("- RAG Database: 42,000+ movies in Pinecone")
print("- Tools: Movie retriever, Library scanner, Torrent manager")

print("\n📊 Database Stats:")
# Simple database query to show scale
sample_movies = movie_retriever_tool._run("popular movies", max_results=5)
print("Sample of available movies:")
display_sample = sample_movies[:300] + "..." if len(sample_movies) > 300 else sample_movies
print(display_sample)

## 🎯 Key Features Demonstrated

This notebook has shown:

1. **Direct Tool Usage**: Testing the RAG retriever without the full workflow
2. **Multi-Agent Routing**: Supervisor directing requests to specialized agents  
3. **Conversation Memory**: Multi-turn conversations with maintained context
4. **Agent Specialization**: Different agents handling different types of tasks
5. **Interactive Testing**: Simple chat interface for live system interaction

The Turtle App demonstrates how LangGraph can orchestrate multiple AI agents to create a sophisticated home theater management system with natural language interaction capabilities.

---

**Next Steps**: 
- Try the interactive chat function above
- Experiment with different movie queries
- Test the conversation memory with related follow-up questions
- Explore the specialized agents for library management and torrent handling